# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia Distances"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-12-09 12:27:19 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package Distances...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Imports

In [ ]:
using Pkg
Pkg.add("Distances")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [2]:
using LinearAlgebra
using Statistics
using Dates
using Distances
using DelimitedFiles

# How to get all the dates corresponding to the month of July 2016?

In [ ]:
dr = Date("2016-07-01"):Day(1):Date("2016-07-31")

Date("2016-07-01"):Day(1):Date("2016-07-31")

In [ ]:
# The above is probably all you need but if if do need to get the dates individually then this works
dr = [Date("2016-07-01"):Day(1):Date("2016-07-31");]

31-element Vector{Date}:
 2016-07-01
 2016-07-02
 2016-07-03
 2016-07-04
 2016-07-05
 2016-07-06
 2016-07-07
 2016-07-08
 2016-07-09
 2016-07-10
 2016-07-11
 2016-07-12
 2016-07-13
 ⋮
 2016-07-20
 2016-07-21
 2016-07-22
 2016-07-23
 2016-07-24
 2016-07-25
 2016-07-26
 2016-07-27
 2016-07-28
 2016-07-29
 2016-07-30
 2016-07-31

# How to compute ((A+B)*(-A/2)) in place (without copy)?

In [ ]:
A = ones(3)
B = 2ones(3)

(A + B) .* (-A / 2)

3-element Vector{Float64}:
 -1.5
 -1.5
 -1.5

# Extract the integer part of a random array of positive numbers?

In [ ]:
Z = rand(0:0.01:100,20)
make_int(x) = trunc(Int,x)
make_int.(Z)

20-element Vector{Int64}:
 31
  8
 51
  2
 67
 67
 96
 27
 23
 55
 90
 62
 80
 79
 72
 96
  2
 99
 81
 66

# Create a 5x5 matrix with row values ranging from 0 to 4 

In [ ]:
A = fill([0:4;], 5)
# This is the solution if you want a array of arrays

5-element Vector{Vector{Int64}}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]
 [0, 1, 2, 3, 4]

In [ ]:
# The solution for this particular problem
A = [0 1 2 3 4]
repeat(A, 5)

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

In [ ]:
# This is a more generic solution but slightly more involved
repeat(reshape([0:4;], 1, :), 5)
# Lets break down as so 
# * [0:4;] creates a vector [0,1,2,3,4], by default Julia makes a column vector
# * reshape([0:4;], 1, :) converts to 1x5 row vector from the column vector
# * The repeat function basically copies it five times

5×5 Matrix{Int64}:
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4
 0  1  2  3  4

# Create a vector of size 10 with values ranging from 0 to 1, both excluded


In [ ]:
Z = [rand(Float64) for i in 1:10]
# Most of the time this will work. I don't really see the need to actually exclude 0 as well from this code but if you do, then

10-element Vector{Float64}:
 0.9478563957814803
 0.3681345922908852
 0.9466226475652395
 0.32830168650621605
 0.13552218264472882
 0.3725138334370284
 0.26594496448031135
 0.4846630817427868
 0.31655588034825044
 0.23482281703174124

In [ ]:
Z = [prevfloat(1.0) * (1 - rand(Float64)) for i in 1:10]
# Again its pretty extremely unlikely you'd actually need it but if you do, then you please read this discussion as well before going in https://discourse.julialang.org/t/how-to-create-a-random-uniform-distribution-between-but-excluding-0-and-10

10-element Vector{Float64}:
 0.8950004928306984
 0.48374977088077004
 0.5743165302063554
 0.494740148104344
 0.766577429100738
 0.1264097879765251
 0.14311948702355523
 0.7060538166200595
 0.7366933075979046
 0.21902266150031433

# Create a random vector of size 10 and sort it

In [ ]:
Z = rand(Int,10)
sort!(Z)

10-element Vector{Int64}:
 -9041140746689117840
 -3093580306631053862
 -2498274159621103025
  -974743244451864736
  -838093373416532481
  1692886258340319378
  3878597274144826153
  4423030593399157736
  7117114196519941441
  8511203450520194029

# Consider two random array A and B, check if they are equal

In [ ]:
A = rand(1:5, 10)
B = rand(1:5, 10)

# To check if the arrays are completely equal
A == B

false

In [ ]:
A = rand(1:5, 10)
B = rand(1:5, 10)
A,B

([4, 5, 3, 2, 3, 4, 1, 4, 5, 4], [3, 5, 4, 1, 5, 5, 4, 3, 2, 5])

In [ ]:
# To check if the elements in the arrays are equal
A .== B

10-element BitVector:
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0

# Consider a random 10x2 matrix representing cartesian coordinates, convert them to polar coordinates


In [ ]:
struct cartesian_point
    X::Float32
    Y::Float32
end
points = [cartesian_point(rand(-900:0.01:900), rand(-900:0.01:900)) for _ = 1:10]

10-element Vector{cartesian_point}:
 cartesian_point(-825.77f0, -415.34f0)
 cartesian_point(564.0f0, -786.35f0)
 cartesian_point(644.91f0, 622.28f0)
 cartesian_point(637.28f0, -607.35f0)
 cartesian_point(-884.21f0, 595.98f0)
 cartesian_point(-344.43f0, -383.8f0)
 cartesian_point(466.89f0, 142.59f0)
 cartesian_point(826.8f0, -31.75f0)
 cartesian_point(503.26f0, 119.27f0)
 cartesian_point(-164.78f0, 390.93f0)

In [ ]:
points = [cartesian_point(rand(), rand()) for _ = 1:10]
Xpoints = [point.X for point in points]
Ypoints = [point.Y for point in points]

10-element Vector{Float32}:
 0.8835904
 0.6172397
 0.2685333
 0.28536692
 0.19637123
 0.75644255
 0.25275177
 0.2707322
 0.3931539
 0.3624934

In [ ]:
radians(x, y) = sqrt(x^2 + y^2)
theta(x, y) = atan(y, x)

theta (generic function with 1 method)

In [ ]:
radians.(Xpoints, Ypoints)
theta.(Xpoints, Ypoints)

10-element Vector{Float32}:
 0.9560654
 1.0856395
 0.430988
 0.5912352
 0.35861602
 0.7840849
 0.9235012
 0.32013956
 0.9817336
 1.5228384

# Create random vector of size 10 and replace the maximum value by 0


In [ ]:
Z = rand(Int8, 10)
print(Z)
Z[argmax(Z)]

Int8[14, -90, -126, 37, -114, -48, -101, 74, 115, -92]

115

# Create a structured array with x and y coordinates covering the [0,1]x[0,1] area


In [ ]:
Z = (X = collect(LinRange(0, 1, 5)), Y = collect(LinRange(0, 1, 5)))

(X = [0.0, 0.25, 0.5, 0.75, 1.0], Y = [0.0, 0.25, 0.5, 0.75, 1.0])

# Given two arrays, X and Y, construct the Cauchy matrix C (Cᵢⱼ = 1 ÷ (xᵢ - yⱼ))

In [ ]:
X = rand(Float32, 10)
Y = rand(Float32, 10)

10-element Vector{Float32}:
 0.97117007
 0.094693065
 0.20137191
 0.44516134
 0.26422274
 0.39019406
 0.13884974
 0.6820953
 0.9912721
 0.35667324

In [ ]:
f(i, j) = 1 ÷ (i - j)
C = (f(x, y) for x in X, y in Y)
det(collect(C))

-5.3118444f13

# How to find the closest value (to a given scalar) in a vector?

In [ ]:
A = rand(Int8, 10)
print(A)
findnearest(A, x) = argmin(abs.(A .- x))

Int8[-20, -111, -119, -112, -26, 36, -63, 84, 101, -46]

findnearest (generic function with 1 method)

In [ ]:
A[findnearest(A,0)]

-20

# Consider a random vector with shape (100,2) representing coordinates, find point by point distances

In [ ]:
A = rand(Int8, (10, 2))
B = rand(Int8, (10, 2))

10×2 Matrix{Int8}:
  -3   -10
 107   -31
  -4   -69
 -58   -41
  74   -65
 -43    46
  17   -89
  83    48
  28   -84
  25  -127

In [ ]:
pairwise(Euclidean(), A, B, dims=1)

10×10 Matrix{Float64}:
 24.2074   0.0       9.16515  4.0       0.0     …   0.0     12.53     25.3969
  0.0      0.0       0.0      8.30662   0.0         0.0      0.0       0.0
 15.5563   0.0       4.0      0.0       0.0         0.0      0.0      13.1529
  0.0      0.0      10.6771   0.0      12.7279      0.0      0.0       0.0
 15.7797   0.0       0.0      0.0       0.0         0.0      0.0      20.0
  6.48074  0.0       0.0      0.0      12.1655  …  15.2971  14.8661    8.544
  0.0      0.0       0.0      9.64365   0.0         0.0      0.0      11.4891
  0.0      0.0       9.16515  0.0       0.0         0.0     16.7631   10.6301
 10.6301   0.0       0.0      0.0      13.4536     11.1803   0.0       0.0
  0.0      6.40312   0.0      0.0      18.3848      0.0      7.54983   0.0

# How to convert a float (32 bits) array into an integer (32 bits) in place

In [ ]:
A = rand(Float32, 10)
trunc.(Int, A)

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

# How to read the following file?
"""
1, 2, 3, 4, 5
6,  ,  , 7, 8
 ,  , 9,10,11
"""

Upload the file here before running

In [ ]:
data = readdlm("num_fle.txt", ',')
data

3×5 Matrix{Any}:
 1     2      3       4   5
 6      "  "   "  "   7   8
  " "   "  "  9      10  11

# What is the equivalent of enumerate for Julia arrays?

In [ ]:
# Enumerate is just enumerate in Julia
Z = [1:9;]
squared = [i^2 for (i, num) in enumerate(Z)]

9-element Vector{Int64}:
  1
  4
  9
 16
 25
 36
 49
 64
 81

# Generate a generic 2D Gaussian-like array

In [ ]:
σ, μ = 1.0, 0.0
X, Y = LinRange(-1, 1, 10), LinRange(-1, 1, 10)
G(x, y, μ, σ) = exp(-(x - μ) .^ 2 / (2.0 * σ^2) - (y - μ) .^ 2 / (2.0 * σ^2))
mat = [G(x, y, μ, σ) for x in X, y in Y]

10×10 Matrix{Float64}:
 0.367879  0.448221  0.519795  0.573753  …  0.519795  0.448221  0.367879
 0.448221  0.546108  0.633313  0.699056     0.633313  0.546108  0.448221
 0.519795  0.633313  0.734444  0.810684     0.734444  0.633313  0.519795
 0.573753  0.699056  0.810684  0.894839     0.810684  0.699056  0.573753
 0.602798  0.734444  0.851723  0.940138     0.851723  0.734444  0.602798
 0.602798  0.734444  0.851723  0.940138  …  0.851723  0.734444  0.602798
 0.573753  0.699056  0.810684  0.894839     0.810684  0.699056  0.573753
 0.519795  0.633313  0.734444  0.810684     0.734444  0.633313  0.519795
 0.448221  0.546108  0.633313  0.699056     0.633313  0.546108  0.448221
 0.367879  0.448221  0.519795  0.573753     0.519795  0.448221  0.367879

# How to randomly place p elements in a 2D array?

In [9]:
n, p = 10, 3
Z = zeros(Int8, n, n)
Z[rand(1:n*n, p)] .= 1
# So this is kind of a hack but I think its okay.

3-element view(::Vector{Int8}, [15, 91, 23]) with eltype Int8:
 1
 1
 1

In [12]:
Z

10×10 Matrix{Int8}:
 0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0

# Subtract the mean of each row of a matrix

In [15]:
Z = rand(1:100, 10, 10)
remove_mean(A) = A .- mean(A)
mapslices(remove_mean, Z, dims = 1)

10×10 Matrix{Float64}:
 -24.4    5.2   22.8    2.1   21.3   26.4   36.0    3.4   10.8  -15.6
   5.6  -17.8  -38.2   15.1   21.3   19.4   16.0  -12.6   41.8   35.4
  14.6   23.2   32.8  -24.9   14.3  -16.6  -26.0   -5.6   -8.2  -22.6
 -33.4   46.2   -1.2  -25.9   28.3  -16.6  -31.0  -21.6   19.8  -23.6
  28.6    8.2    1.8   39.1  -24.7  -39.6    6.0   -8.6  -43.2   23.4
 -31.4  -19.8  -41.2   12.1  -58.7   30.4   23.0   24.4   44.8    5.4
   4.6   40.2  -49.2   32.1  -10.7   -4.6  -36.0   28.4   19.8  -14.6
  21.6  -15.8    2.8  -27.9   -4.7  -12.6   -2.0  -36.6  -46.2   28.4
  -1.4  -46.8   42.8  -43.9   31.3    9.4    9.0   18.4   -6.2  -46.6
  15.6  -22.8   26.8   22.1  -17.7    4.4    5.0   10.4  -33.2   30.4

# How to sort an array by the nth column?

In [19]:
Z = rand(1:100, 10, 10)
nᵗʰ = 3
sort(Z, by = Z[:,nᵗʰ],dims=1)

LoadError: ignored